In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [175]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import EditedNearestNeighbours,RandomUnderSampler
from sklearn.kernel_approximation import Nystroem
from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline,make_pipeline
from imblearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
#model
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
os.chdir('C:\\Users\\ACER\\IBM Skill networks lab\\Datasets')

In [4]:
from datasetup import churndata,colors,palette,fulldata

In [5]:
sns.set_palette(palette)

In [6]:
churndata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7043 non-null   object 
 1   months         7043 non-null   int64  
 2   offer          7043 non-null   object 
 3   phone          7043 non-null   object 
 4   multiple       7043 non-null   object 
 5   internet_type  7043 non-null   object 
 6   gb_mon         7043 non-null   int64  
 7   security       7043 non-null   object 
 8   backup         7043 non-null   object 
 9   protection     7043 non-null   object 
 10  support        7043 non-null   object 
 11  unlimited      7043 non-null   object 
 12  contract       7043 non-null   object 
 13  paperless      7043 non-null   object 
 14  payment        7043 non-null   object 
 15  monthly        7043 non-null   float64
 16  total_revenue  7043 non-null   float64
 17  satisfaction   7043 non-null   int64  
 18  churn_va

In [205]:
df['protection']

0       Yes
1        No
2        No
3       Yes
4        No
       ... 
7038     No
7039    Yes
7040    Yes
7041     No
7042    Yes
Name: protection, Length: 7043, dtype: object

In [7]:
churndata['contract'].value_counts()

Month-to-Month    3610
Two Year          1883
One Year          1550
Name: contract, dtype: int64

In [9]:
churndata.head()

,id,months,offer,phone,multiple,internet_type,gb_mon,security,backup,protection,...,unlimited,contract,paperless,payment,monthly,total_revenue,satisfaction,churn_value,churn_score,cltv
0,8779-QRDMV,1,None,No,No,DSL,8,No,No,Yes,...,No,Month-to-Month,Yes,Bank Withdrawal,39.65,59.65,3,1,91,5433
1,7495-OOKFY,8,Offer E,Yes,Yes,Fiber Optic,17,No,Yes,No,...,Yes,Month-to-Month,Yes,Credit Card,80.65,1024.10,3,1,69,5302
2,1658-BYGOY,18,Offer D,Yes,Yes,Fiber Optic,52,No,No,No,...,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1910.88,2,1,81,3179
3,4598-XLKNJ,25,Offer C,Yes,No,Fiber Optic,12,No,Yes,Yes,...,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,2995.07,2,1,88,5337
4,4846-WHAFZ,37,Offer C,Yes,Yes,Fiber Optic,14,No,No,No,...,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,3102.36,2,1,67,2793


In [10]:
df = churndata.drop(['id', 'phone', 'total_revenue', 'cltv', 'churn_score'],axis=1).copy()

In [25]:
df.shape

(7043, 16)

In [12]:
round(df.describe(),2)

,months,gb_mon,monthly,satisfaction,churn_value
count,7043.00,7043.00,7043.00,7043.00,7043.00
mean,32.39,20.52,64.76,3.24,0.27
std,24.54,20.42,30.09,1.20,0.44
min,1.00,0.00,18.25,1.00,0.00
25%,9.00,3.00,35.50,3.00,0.00
50%,29.00,17.00,70.35,3.00,0.00
75%,55.00,27.00,89.85,4.00,1.00
max,72.00,85.00,118.75,5.00,1.00


In [14]:
df.describe(include='object').T

,count,unique,top,freq
offer,7043,6,None,3877
multiple,7043,2,No,4072
internet_type,7043,4,Fiber Optic,3035
security,7043,2,No,5024
backup,7043,2,No,4614
protection,7043,2,No,4621
support,7043,2,No,4999
unlimited,7043,2,Yes,4745
contract,7043,3,Month-to-Month,3610
paperless,7043,2,Yes,4171


In [28]:
categorical = list(df.columns[df.dtypes == np.object])
categorical

['offer',
 'multiple',
 'internet_type',
 'security',
 'backup',
 'protection',
 'support',
 'unlimited',
 'contract',
 'paperless',
 'payment']

In [29]:
ordinal = ['contract','satisfaction']
categorical.remove('contract')

In [44]:
numerical = list(df.columns[(df.dtypes == np.int64) | (df.dtypes == np.float64)])
numerical.remove('churn_value')
numerical.remove('satisfaction')
categorical.extend(['churn_value'])

numerical

['months', 'gb_mon', 'monthly']

In [24]:
len(df.columns)

16

In [45]:
#no missing value
df.isnull().sum()

months           0
offer            0
multiple         0
internet_type    0
gb_mon           0
security         0
backup           0
protection       0
support          0
unlimited        0
contract         0
paperless        0
payment          0
monthly          0
satisfaction     0
churn_value      0
dtype: int64

In [49]:
# cannot continue process duplicate value
#df.loc[df.duplicated()]

In [54]:
def variable_stats(column,dataframe):
    """
    how to use
    
    q1,q2,q3,iqr,low_limit,up_limit,outliers = variable_stats(column,dataframe)
    """
    q1 = np.percentile(dataframe[column],25)
    q2 = np.percentile(dataframe[column],50)
    q3 = np.percentile(dataframe[column],75)
    iqr = q3-q1
    
    low_limit = q1 - 3*iqr
    up_limit = q3 + 3*iqr
    outliers = dataframe.loc[(dataframe[column] > up_limit) | (dataframe[column] < low_limit)]
    
    return q1,q2,q3,iqr,low_limit,up_limit,outliers

In [55]:
# no outliers detected
for i in numerical:
    q1,q2,q3,iqr,low_limit,up_limit,outliers = variable_stats(i,df)
    display(outliers)

,months,offer,multiple,internet_type,gb_mon,security,backup,protection,support,unlimited,contract,paperless,payment,monthly,satisfaction,churn_value


,months,offer,multiple,internet_type,gb_mon,security,backup,protection,support,unlimited,contract,paperless,payment,monthly,satisfaction,churn_value


,months,offer,multiple,internet_type,gb_mon,security,backup,protection,support,unlimited,contract,paperless,payment,monthly,satisfaction,churn_value


In [56]:
# class balance check
df['churn_value'].value_counts()
#skewed class need proper rebalancing

0    5174
1    1869
Name: churn_value, dtype: int64

In [ ]:
#preprocess with sklearn pipeline

In [73]:
unique_val_ord = [(i, df[i].unique()) for i in df[ordinal]]
unique_val_ord

[('contract', array(['Month-to-Month', 'One Year', 'Two Year'], dtype=object)),
 ('satisfaction',
  [3, 2, 1, 4, 5]
  Categories (5, int64): [3, 2, 1, 4, 5])]

In [76]:
#change column type
df['satisfaction'] = df['satisfaction'].astype('category')
ord_val_satis = [1,2,3,4,5]
ord_val_cont = ['Month-to-Month', 'One Year', 'Two Year']
categorical.remove('churn_value')
categorical

['offer',
 'multiple',
 'internet_type',
 'security',
 'backup',
 'protection',
 'support',
 'unlimited',
 'paperless',
 'payment']

In [152]:
ordinal
categorical
numerical

num_transform = Pipeline(steps=[
    ('num_impute',SimpleImputer(strategy='most_frequent'))
])

cat_transform = Pipeline(steps=[
    ('cat_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='error',drop='first'))
])

ord_transform = Pipeline(steps=[
    ('ord_impute', SimpleImputer(strategy='most_frequent')),
    ('ord_ec',OrdinalEncoder(categories=[ord_val_cont,ord_val_satis],
                             handle_unknown='ignore'))
])

preprocess = ColumnTransformer(transformers=[
    ('cat',cat_transform,categorical),
    ('num',num_transform,numerical),
    ('ord', ord_transform, ordinal)
])

model_pipe = Pipeline(steps=[
    ('preprocess',preprocess),
    ('scale',StandardScaler()),
    ('upsample',SMOTE(sampling_strategy={1:3000},random_state=42,)),
    ('downsample',RandomUnderSampler(sampling_strategy={0:3000},random_state=42)),
    ('kr_appx',Nystroem()),
    ('svm',SVC())
])

In [92]:
X = df.drop('churn_value',axis=1)
y = df['churn_value']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=700,
                                                    random_state=42,
                                                   stratify=y)

In [89]:
X_train.shape

(6343, 15)

In [131]:
model_pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['offer', 'multiple',
                                                   'internet_type', 'security',
                                                   'backup', 'protection',
                                                   'support', 'unlimited',
                                                   'paperless', 'payment']),
                                                 ('num',
                                                  Pipeline(steps=[('num_impute',
                                

In [132]:
model_pipe.named_steps['upsample'].get_params()

{'k_neighbors': 5,
 'n_jobs': None,
 'random_state': 42,
 'sampling_strategy': {1: 3000}}

In [153]:
param_grid = {'svm__kernel':np.array(['rbf','polynomial']),
             'svm__degree':np.array([2,4,6,8])}
svm_search = GridSearchCV(model_pipe,param_grid=param_grid)
svm_search.fit(X_train,y_train)

D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\imblearn\pipeline.py", line 266, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\svm\_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\svm\_base.py", line 277, in _dense_fit
    self._probB, self.fit_status_ = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx", line 176, in sklearn.svm._libsvm.fit
ValueError: 'polynomial' is not in list

  warnin

GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('cat_impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['offer',
                                                                          'multiple',
                                                                          'internet_type',
                                                                          'security',
                                                                 

In [206]:
svm_search.best_params_
#svm_search.best_score_

{'svm__degree': 8, 'svm__kernel': 'rbf'}

In [159]:
gs_svm_yhat = svm_search.predict(X_test)
print(classification_report(y_test,gs_svm_yhat))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       514
           1       0.88      0.91      0.90       186

    accuracy                           0.94       700
   macro avg       0.93      0.93      0.93       700
weighted avg       0.94      0.94      0.94       700



In [137]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.metrics import classification_report,plot_confusion_matrix
from joblib import load,dump

In [133]:
y_pred_svm = model_pipe.predict(X_test)

In [162]:
pwd

'C:\\Users\\ACER\\IBM Skill networks lab\\Datasets'

In [161]:
dump(svm_search,'svm_model_pipeline.joblib')

['svm_model_pipeline.joblib']

In [ ]:
from

In [170]:
ordinal
categorical
numerical

num_transform = Pipeline(steps=[
    ('num_impute',SimpleImputer(strategy='most_frequent'))
])

cat_transform = Pipeline(steps=[
    ('cat_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='error',drop='first'))
])

ord_transform = Pipeline(steps=[
    ('ord_impute', SimpleImputer(strategy='most_frequent')),
    ('ord_ec',OrdinalEncoder(categories=[ord_val_cont,ord_val_satis],
                             handle_unknown='ignore'))
])

preprocess = ColumnTransformer(transformers=[
    ('cat',cat_transform,categorical),
    ('num',num_transform,numerical),
    ('ord', ord_transform, ordinal)
])

model_pipe = Pipeline(steps=[
    ('preprocess',preprocess),
    ('scale',StandardScaler()),
    ('upsample',SMOTE(sampling_strategy={1:3000},random_state=42,)),
    ('downsample',RandomUnderSampler(sampling_strategy={0:3000},random_state=42)),
    ('kr_appx',Nystroem()),
    ('logreg',LogisticRegression(max_iter=1000))
])

In [171]:
param_grid = {'logreg__penalty':np.array(['none','elasticnet']),
             'logreg__solver':np.array(['saga']),
             'logreg__C':np.array([0.01,0.5,1,3,5]),
             'logreg__l1_ratio':np.linspace(0,1,5)}
logistic_search = GridSearchCV(model_pipe,param_grid=param_grid)
logistic_search.fit(X_train,y_train)

D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=none)
  warnings.warn("l1_ratio parameter is only used when penalty is "
D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
D:\conda\envs\Kevin X DS\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not conv

GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('cat_impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['offer',
                                                                          'multiple',
                                                                          'internet_type',
                                                                          'security',
                                                                 

In [172]:
logistic_search.best_params_

{'logreg__C': 5.0,
 'logreg__l1_ratio': 0.5,
 'logreg__penalty': 'none',
 'logreg__solver': 'saga'}

In [190]:
logistic_search_yhat = logistic_search.predict(X_test)
print(classification_report(y_test,logistic_search_yhat))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       514
           1       0.86      0.90      0.88       186

    accuracy                           0.93       700
   macro avg       0.91      0.92      0.92       700
weighted avg       0.94      0.93      0.93       700



In [174]:
dump(logistic_search,'logistic_regression_model_pipeline.joblib')

['logistic_regression_model_pipeline.joblib']

In [213]:
ordinal
categorical
numerical
dtree = DecisionTreeClassifier(max_depth=1,max_features='sqrt')
num_transform = Pipeline(steps=[
    ('num_impute',SimpleImputer(strategy='most_frequent'))
])

cat_transform = Pipeline(steps=[
    ('cat_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='error',drop='first'))
])

ord_transform = Pipeline(steps=[
    ('ord_impute', SimpleImputer(strategy='most_frequent')),
    ('ord_ec',OrdinalEncoder(categories=[ord_val_cont,ord_val_satis],
                             handle_unknown='ignore'))
])

preprocess = ColumnTransformer(transformers=[
    ('cat',cat_transform,categorical),
    ('num',num_transform,numerical),
    ('ord', ord_transform, ordinal)
])

model_pipe = Pipeline(steps=[
    ('preprocess',preprocess),
    ('scale',StandardScaler()),
    ('upsample',SMOTE(sampling_strategy={1:3000},random_state=42,)),
    ('downsample',RandomUnderSampler(sampling_strategy={0:3000},random_state=42)),
    #('kr_appx',Nystroem()),
    ('abc',AdaBoostClassifier(base_estimator=dtree,
                              random_state=42,
                             learning_rate=1))
])

In [214]:
param_grid = {'abc__n_estimators':np.array([50,100,150,200,300])}
adac_model = GridSearchCV(model_pipe,param_grid=param_grid)
adac_model.fit(X_train,y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('cat_impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(drop='first'))]),
                                                                         ['offer',
                                                                          'multiple',
                                                                          'internet_type',
                                                                          'security',
                                                                 

In [215]:
adac_model.best_params_
adac_model.best_score_

0.9463973470752449

In [216]:
adac_search_yhat = adac_model.predict(X_test)
print(classification_report(y_test,adac_search_yhat))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96       514
           1       0.88      0.92      0.90       186

    accuracy                           0.95       700
   macro avg       0.93      0.94      0.93       700
weighted avg       0.95      0.95      0.95       700



In [302]:
adac_model.best_estimator_.named_steps.abc.feature_importances_
trnsfm_cat = adac_model.best_estimator_.named_steps.preprocess.transformers_[0][1].named_steps.ohe.get_feature_names(categorical)

all_var = []
all_var.extend(list(trnsfm_cat))
all_var.extend(list(numerical))
all_var.extend(list(ordinal))
f__importances = list(adac_model.best_estimator_.named_steps.abc.feature_importances_)
features_complete = []

for i,j in zip(all_var,f__importances):
    features_complete.append(pd.Series([j],name=i))

features_complete = pd.DataFrame(features_complete)
features_complete
#i think the methods works
# the data is messed up by sampler

,0
offer_Offer A,0.013333
offer_Offer B,0.026667
offer_Offer C,0.033333
offer_Offer D,0.020000
offer_Offer E,0.020000
multiple_Yes,0.060000
internet_type_DSL,0.013333
internet_type_Fiber Optic,0.033333
internet_type_None,0.033333
security_Yes,0.020000


In [296]:
adac_model.best_estimator_.named_steps.abc.feature_importances_

array([0.01333333, 0.02666667, 0.03333333, 0.02      , 0.02      ,
       0.06      , 0.01333333, 0.03333333, 0.03333333, 0.02      ,
       0.01333333, 0.03333333, 0.03333333, 0.03333333, 0.06666667,
       0.04666667, 0.02      , 0.08      , 0.12666667, 0.15333333,
       0.10666667, 0.01333333])

In [187]:
dump(adac_model,'adaboost_classifier_pipeline_model.')

['adaboost_classifier_pipeline_model.joblib']

In [285]:
labels = ['SVM','Logistic Regression','Adaboost']
models = [svm_search,logistic_search,adac_model]
metrics = []
ypred_all = []
for i,j in zip(labels,models):
    ypred_all.append(j.predict(X_test))
for k in range(len(ypred_all)):
    prec,rec,f1,_ = precision_recall_fscore_support(y_test,ypred_all[k],average='weighted')
    acc = accuracy_score(y_test,ypred_all[k])
    auc = roc_auc_score(y_test,ypred_all[k],average='weighted')
    metrics.append(pd.Series({
        'precision':prec,
        'recall':rec,
        'f1_score':f1,
        'accuracy':acc,
        'auc':auc
    },name=labels[k]))
metrics 

[precision    0.944870
 recall       0.944286
 f1_score     0.944519
 accuracy     0.944286
 auc          0.932900
 Name: SVM, dtype: float64,
 precision    0.935447
 recall       0.934286
 f1_score     0.934720
 accuracy     0.934286
 auc          0.922660
 Name: Logistic Regression, dtype: float64,
 precision    0.948303
 recall       0.947143
 f1_score     0.947534
 accuracy     0.947143
 auc          0.939992
 Name: Adaboost, dtype: float64]

In [199]:
metrics

,SVM,Logistic Regression,Adaboost
precision,0.944870,0.935447,0.948303
recall,0.944286,0.934286,0.947143
f1_score,0.944519,0.934720,0.947534
accuracy,0.944286,0.934286,0.947143
auc,0.932900,0.922660,0.939992


# Mini Experiment to determine ordinal order on ordinal encoder 

In [56]:
oe = OrdinalEncoder(categories=[['Two Year','Month-to-Month','One Year']])

In [54]:
contract = np.array(churndata['contract'])

In [57]:
ordinal_example = oe.fit_transform(contract.reshape(-1,1))
ordinal_example

array([[1.],
       [1.],
       [1.],
       ...,
       [2.],
       [1.],
       [0.]])

In [58]:
churndata['contract'].value_counts()

Month-to-Month    3610
Two Year          1883
One Year          1550
Name: contract, dtype: int64

In [59]:
ordinal_example = pd.DataFrame(ordinal_example)
ordinal_example.value_counts()

1.0    3610
0.0    1883
2.0    1550
dtype: int64

# case closed

In [3]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
adac = AdaBoostClassifier()